## Clip Maps Against Districts
* Look at the maps using `find difference and clip`.

In [ ]:
import dask.dataframe as dd
import dask_geopandas as dg
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely.wkt
import A1_provider_prep
import A2_analysis
import A3_other
from calitp.sql import to_snakecase
from shared_utils import geography_utils, utils

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
# verizon = A1_provider_prep.load_verizon()

In [ ]:
unique_routes = A3_other.load_unique_routes_df()

In [ ]:
# Using finding diff and clipping
att = A1_provider_prep.load_att()

In [ ]:
att.geometry.is_valid.all()

In [ ]:
# Original ATT map clipped to CA
att_original = gpd.read_parquet(f"{A1_provider_prep.GCS_FILE_PATH}att_ca_only.parquet").plot()

In [ ]:
tmobile = A1_provider_prep.load_tmobile()

In [ ]:
# tmobile.plot()

In [ ]:
one_dist_routes, multi_dist_routes, all_routes = A3_other.find_multi_district_routes()

In [ ]:
# Maps created with clip/difference doesnt work
# Gives the error: TopologyException: side location conflict at -118.62460299999999 35.030214999999998. This can occur if the input geometry is invalid.
# However att.geometry.is_valid.all() returns true. 
# multi_dist_routes_dissolved.geometry.is_valid.all() also returns true
# switched back to sjoin with districts version of ATT 
att_o = gpd.overlay(multi_dist_routes_dissolved, att, how="intersection", keep_geom_type=False
    )

In [ ]:
# Create a new route length for portions covered by cell coverage
# Why am I getting percentaes that are like 200% more than the original route length?
att_o= att_o.assign(
        route_length=att_o.geometry.to_crs(geography_utils.CA_StatePlane).length
    )
    
att_o = att_o.drop_duplicates().reset_index(drop = True)

In [ ]:
att_o = A2_analysis.dissolve_summarize(att_o)

In [ ]:
att_o_routes = att_o.long_route_name.unique().tolist()

In [ ]:
unique_routes = A3_other.load_unique_routes_df()

In [ ]:
att_o_routes_original_geometry = unique_routes[unique_routes.long_route_name.isin(att_o_routes)]

In [ ]:
a = att.explore(
    tiles = "CartoDB positron",
    width=800,
    height=400,
)

In [ ]:
# Clipped geometry
a = att_o_routes_original_geometry.explore("long_route_name",
    m=a, legend = False
)

In [ ]:
a

In [ ]:
# att_o.drop(columns = ["geometry"]).sort_values('agency')

In [ ]:
#  Put unique routes on the left and the provider on the right
tmobile_o = A2_analysis.comparison(multi_dist_routes_dissolved, tmobile)

In [ ]:
tmobile_o = A2_analysis.dissolve_summarize(tmobile_o)

In [ ]:
n = tmobile.explore(
    tiles = "CartoDB positron",
    width=800,
    height=400,
)

In [ ]:
# n

In [ ]:
# att_o.shape

In [ ]:
# att_clipped_overlay = A2_analysis.overlay_single_routes(att_clipped,multi_dist_routes_dissolved,'att_clipped')